## **CHAPTER 7 데이터 준비하기: 다듬기, 변형, 병합**

**데이터 분석과 모델링 작업은 데이터를 불러오고, 다듬고, 변형하고 재정렬하는, 데이터 준비 과정에 많은 시간이 소요된다.**

**가끔은 파일이나 데이터베이스에 저장된 데이터가 애플리케이션에서 사용하기 쉽지 않은 방식으로 저장되어 있을 때도 있다.**

**파이썬 표준 라이브러리와 pandas를 함께 사용하면 큰 어려움 없이 데이터를 원하는 형태로 가공할 수 있다.**

## **7.1 데이터 합치기**

**pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해서 합칠 수 있다.**

> **pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터베이스의 join과 유사하다.**

> **pandas.concat은 하나의 축을 따라 객체를 이어붙인다.**

> **conbine_first 인스턴스 메서드는 두 객체를 포개서 한 객체에 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.**

### **7.1.1 데이터베이스 스타일로 DataFrame 합치기**

**머지(merge)나 조인(join) 연산은 관계형 데이터베이스의 핵심적인 연산으로, 키를 하나 이상 사용해서 데이터 집합의 로우를 합친다.**

**pandas의 merge 함수를 이용해서 이런 알고리즘을 데이터에 적용할 수 있다.**

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

df1 = DataFrame({'key':['b','b','a','c','a','a','b'], 'data1':range(7)})
df2 = DataFrame({'key':['a','b','d'],'data2':range(3)})

In [2]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [4]:
# 일 대 다의 예제다. df1 데이터는 key 칼럼에 여러 개의 a,b를 가지고 있고 df2의 key 칼럼은 유일한 로우를 가지고 있다.
# 이 객체에 대해 merge 함수를 호출하면 다음과 같은 결과를 얻을 수 있다.

# 여기서 어떤 칼럼을 병합할 것인지 명시하지 않았는데, merge 함수는 겹치는 칼러의 키로 사용한다.
# df1의 key와 df2의 key의 중복 값만을 merge해준다.
pd.merge(df1,df2) 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [5]:
# 하지만 명시적으로 지정하는 습관을 들리는게 좋다.
df3 = pd.merge(df1,df2,on='key') # on='공통되는 칼럼명'
df3

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [6]:
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],
                'data1':range(7)})
df4 = DataFrame({'rkey':['a','b','d'],'data2':range(3)})

In [7]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [8]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [9]:
# 만약 두 객체에 공통되는 칼럼 이름이 하나도 없다면 따로 지정해주면 된다.
pd.merge(df3, df4, left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


**merge 함수는 기본적으로 내부조인(inner join)을 수행하여 교집합인 결과를 반환한다.**

> **how 인자로 'left','right','outer'를 넘겨서 각각 왼쪽 우선 외부조인, 오른쪽 우선 외부조인, 완전 외부조인을 수행할 수도 있다.**

> **완전 외부조인은 합집합인 결과를 반환하고 왼쪽 우선 외부조인과 오른쪽 우선 외부조인은 각각 왼쪽 혹은 로우쪽의 모든 로우를 포함하는 결과를 반환**

In [10]:
pd.merge(df1,df2,how='outer') # 완전 외부조인으로 합집합인 결과를 반환

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [11]:
# 다 대 다 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다.
df1 = DataFrame({'key':['b','b','a','c','a','b'],
                 'data1':range(6)})

df2 = DataFrame({'key':['a','b','a','b','d'],
                 'data2':range(5)})

In [12]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [13]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [14]:
# 다 대 다 조인은 두 로우의 데카르트 곱을 반환한다.
# df1에는 3개의 'b'로우가 있고 오른쪽에는 2개의 'b'로우가 있는데, 결과는 6개의 'b'로우가 된다.
# b(df1)=0, b(df2)=1
# b(df1)=0, b(df2)=3
# b(df1)=1, b(df2)=1
# b(df1)=1, b(df2)=3
# b(df1)=5, b(df2)=1
# b(df1)=5, b(df2)=3

pd.merge(df1, df2, on='key',how='left') # 공통 칼럼을 key로 지정하고, 왼쪽(df1)을 우선 조인한다.

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [15]:
pd.merge(df1,df2, how='inner') # 왼쪽 우선 조인과 다른점은 inner는 두개의 공통값만을 데카르트 곱으로 반환한다는 점이다.

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [16]:
# 여러 개의 키를 병합하려면 칼럼 이름이 들어간 리스트를 넘기면 된다.
left = DataFrame({'key1':['foo','foo','bar'],
                  'key2':['one','two','one'],
                  'lval':[1,2,3]})
right = DataFrame({'key1':['foo','foo','bar','bar'],
                   'key2':['one','one','one','two'],
                   'rval':[4,5,6,7]})

In [17]:
# merge 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알려면 실제 구현과는 조금 다르지만,
# 여러 개의 키가 들어있는 튜플의 배열이 단일 조인키로 사용된다고 생각하면 된다.
pd.merge(left,right,on=['key1','key2'],how='outer') # 완전 외부조인을 통해 합집한인 결과를 반환

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


**머지 연산에서 고려해야 할 마지막 사항은 겹치는 칼럼 이름에 대한 처리다.**

> **축의 이름을 변경해서 수동으로 칼럼 이름을 겹치게 할 수도 있고**

> **merge 함수에 있는 suffixes 인자를 통해 두 DataFrame 객체에서 겹치는 칼럼 이름 뒤에 붙일 문자열을 지정해 줄 수도 있다.**

In [18]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [19]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


|  인자  | 설명 | 
|:---:|---:|
| `left` | 머지하려는 DataFrame 중 왼쪽에 위치한 DataFrame | 
| `right`| 머지하려는 DataFrame 중 오른쪽에 위치한 DataFrame | 
|  `how` | 조인 방법. 'inner', 'outer', 'left', 'right', 기본 값은 'inner'  | 
|  `on`  | 조인하려는 로우 이름. 반드시 두 DataFrame 객체 모두에 있는 이름이어야 한다. 만약 명시되지 않고 다른 조인 키도 주어지지 않으면 left와 right에서 공통되는 칼럼을 조인 키로 사용한다. |
| `left_on` | 조인 키로 사용할 left DataFrame의 칼럼 | 
| `right_on`| 조인 키로 사용할 right DataFrame의 칼럼 | 
| `left_index` | 조인 키로 사용할 left DataFrame의 색인 로우 (다중 색인일 경우의 키) | 
| `right_index` | 조인 키로 사용할 right DataFrame의 색인 로우 (다중 색인일 경우의 키) | 
| `sort` | 조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본 값은 True. 대용량 데이터의 경우 False라면 성능상의 이득을 얻을 수도 있다. | 
| `suffixes` | 칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플 기본 값 | 
| `copy` | False일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본 값은 항상 복사가 이루어진다. |  

### **7.1.2 색인 머지하기**

**머지하려는 키가 DataFrame의 색인일 수 있다. 이때에는 left_index=True 혹은 right_index=True 옵션을 지정해 해당 색인을 머지 키로 사용할 수 있다.**

In [20]:
left1 = DataFrame({'key':['a','b','a','a','b','c'],
                   'value':range(6)})

right1 = DataFrame({'group_val':[3.5,7]}, index=['a','b'])

In [21]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [22]:
right1

,group_val
a,3.5
b,7.0


In [23]:
pd.merge(left1,right1,left_on='key',right_index=True) # right_index=True를 통해 a(index=0),b(index=1)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [24]:
# 머지는 기본적으로 교집합을 구하지만 외부조인을 실행해서 합집합을 구할 수도 있다.
pd.merge(left1,right1,left_on='key',right_index=True,how='outer') # 합집합이기 때문에 c가 추가

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [25]:
# 계층 index된 데이터는 약간 복잡하다.
lefth = DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                   'key2':[2000,2001,2002,2001,2002],
                   'data':np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6,2)),
                  index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                         [2001,2000,2000,2000,2001,2002]],
                  columns=['event1','event2'])

In [26]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [27]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [28]:
# 이런 경우에는 리스트로 여러 개의 칼럼을 지정해서 머지해야 한다.(중복되는 색인 값을 다룰때는 주의하자.)
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [29]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer') # 완전 외부 조인으로 변경

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [30]:
# 양쪽에 공통으로 있는 여러 개의 index을 머지하는 것도 가능하다.
left2 = DataFrame([[1.,2.],[3.,4.],[5.,6.]],index=['a','c','e'],columns=['Ohio','Nevada'])
right2 = DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],
                  index=['b','c','d','e'],columns=['Missouri','Albama'])

In [31]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [32]:
right2

,Missouri,Albama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [33]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Albama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [34]:
# index로 merge할 때 DataFrame의 join 메서드를 사용하면 편리하다.
# join 메서드는 칼럼이 겹치지 않으며 완전히 같거나 유사한 index 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다.
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Albama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [35]:
# 과거에 작성된 pandas의 일부 코드 제약으로 인해 DataFrame의 join 메서드는 왼쪽 우선 조인을 수행한다.
# join 메서드를 호출한 DataFrame의 칼럼 중 하나에 대해서 조인을 수행하는 것도 가능하다.
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [36]:
right1

,group_val
a,3.5
b,7.0


In [39]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [43]:
# index 대 index으로 두 DataFrame을 합치려면 그냥 간단히 merge하려는 DataFrame의 리스트를 join 메서드에 넘기면 된다.
# 하지만 일반적으로 이런 merge는 다음의 concat 메서드를 사용한다.
another = DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                   index=['a','c','e','f'],columns=['New York','Oregon'])

In [44]:
right2

,Missouri,Albama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [45]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [46]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Albama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [47]:
left2.join([right2,another],how='outer')

,Ohio,Nevada,Missouri,Albama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### **7.1.3 축 따라 이어붙이기**

**데이터를 합치는 또 다른 방법으로는 이어붙이기(concatenation),연결(binding),적층(stacking)등이 있는데, Numpy는 ndarray를 연결하는 concatenate 함수를 제공한다.**

In [48]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [49]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

**Series나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 이어붙일 수 있도록 되어 있다. 특히 다음 사항을 고려해야 한다.**

> **만약 연결하려는 두 객체의 index가 서로 다르다면 결과는 그 index의 교집합이어야 하는가 아니면 합집합이어야 하는가?**

> **합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있는가?**

> **어떤 축으로 연결할 것이지 고려해야 하는가?**

In [50]:
# pandas의 concat 함수는 이 사항에 대한 해당을 제공한다.
# 앞으로 concat 함수가 어떻게 동작하는지 다양한 예제를 통해 알아본다.
# index가 겹치지 않는 3개의 Series를 생성한다.
s1 = Series([0,1],index=['a','b'])
s2 = Series([2,3,4],index=['c','d','e'])
s3 = Series([5,6],index=['f','g'])

In [51]:
# 이 세 객체를 리스트로 묶어서 concat 함수에 전달하면 값과 index를 알려준다.
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [53]:
# concat 함수는 axis=0을 기본 값으로 하고 새로운 Series 객체를 생성한다.
# 만약 axis=1을 넘긴다면 결과는 Series가 아니라 DataFrame이 될 것이다.(axis=1은 칼럼을 의미한다.)
pd.concat([s1,s2,s3],axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [55]:
# 겹치는 축이 없기 때문에 외부조인(outer 메서드)으로 정렬된 합집합을 얻었지만 join='inner'를 넘겨서 교집합을 구할 수도 있다.
s4 = pd.concat([s1*5,s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [56]:
pd.concat([s1,s4],axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [57]:
pd.concat([s1,s4],axis=1,join='inner') # join = 'inner'를 넘겨서 두 Series 객체의 교집합을 구할 수 있다.

,0,1
a,0,0
b,1,5


In [58]:
# join_axes 인자로 merge하려는 축을 직접 지정해줄 수도 있다.
pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [59]:
# Series를 이어붙이기 전의 개별 Series를 구분할 수 없다는 문제가 생기는데, 이 경우 이어붙인 축에 대해 계층적 index을 생성하여 식별이 가능하도록 할 수 있다.
# 계층적 index을 생성하려면 keys 인자를 사용하면 된다.
result = pd.concat([s1,s1,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [60]:
result.unstack() # unstack 함수에 대한 자세한 내용은 나중에 알아본다.

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [61]:
# Series는 axis=1로 병합할 경우 keys는 DataFrame의 칼럼 제목이 된다.
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [62]:
# DataFrame 객체에 대해서도 지금까지와 같은 방식으로 적용할 수 있다.
df1 = DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])
df2 = DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])

In [63]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [64]:
df2

,three,four
a,5,6
c,7,8


In [66]:
pd.concat([df1,df2], axis=1, keys=['level1','level2']) # 왼쪽 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0